In [1]:
# Import libraries
import logging
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import logging
from itertools import chain
import json
import boto3

In [6]:
import threading


class MultiThreading(object):

    def __init__(self, scrapers):
        self.scrapers = scrapers

    def run(self):
        threads = []

        for i in range(len(self.scrapers)):
            t = threading.Thread(target=self.scrapers[i].start)
            t.start()
            threads.append(t)

        for thread in threads:
            thread.join()


In [7]:
class Scraper(object):
    """
    Pull player info down from web
    """

    # Initialize array to store player
    players_scraped = []

    # Instantiate scraper
    def __init__(self, urls):
        self.urls = urls
        self.logger = logging.getLogger("sLogger")

    # request to get the url
    def get_page(self, url):
        response = requests.get(url)
        if response.status_code:
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.find("tbody", {"class": "list"})
        else:
            self.logger.error("Error" + response.status_code)
            return None

    # helper method to get players
    def get_players(self,trs):
        return [extract_info(tr) for tr in trs]
    
    # helper method to get stats
    def get_stats(self,trs):
        return [extract_stats(tr) for tr in trs]

    # method to extract and copy player info from web
    def scrap(self, urls):
        for url in urls:
            tbody = self.get_page(url)
            if tbody is None:
                continue
            trs = tbody.findAll("tr")
            Scraper.players_scraped.append(self.get_players(trs))
            self.logger.info("Page{} scraped".format(len(Scraper.players_scraped)))
    # method to start the scraper
    def start(self):
        self.scrap(self.urls)

In [8]:

url = "https://sofifa.com/players?offset="
urls = [url + str(offset) for offset in range(0,120, 60)]
# Parameters
number_of_scraper = 5
pages = 10

scrapers = [Scraper(urls[pages * i:min(pages * (i + 1), len(urls))]) 
            for i in range(number_of_scraper)]
multi_threading = MultiThreading(scrapers)

#     multi_threading.run()

In [9]:
def flatten(d):
    out = {}
    for key, val in d.items():
        if isinstance(val, dict):
            val = [val]
        if isinstance(val, list):
            for subdict in val:
                deeper = flatten(subdict).items()
                out.update({key2: val2 for key2, val2 in deeper})
        else:
            out[key] = val
    return out

In [10]:
t1 = time.time()
multi_threading.run()

array_players = Scraper.players_scraped
players = list(map(flatten, (chain.from_iterable(Scraper.players_scraped))))

df_multi_thread = pd.DataFrame(players)
df_multi_thread.drop_duplicates(ignore_index = True)
print("Multi threading time taken: ", time.time() - t1)
df_multi_thread.head()


Multi threading time taken:  63.276745080947876


,name,country,age,overall,potential,club,best_position,value,wage,preferred_foot,...,Vision,Penalties,Composure,Defensive Awareness,Standing Tackle,Sliding Tackle,Diving,Handling,Kicking,Reflexes
0,Roy Krishna,Fiji,33,71,71,ATK Mohun Bagan FC,ST,€1.2M,€3K,Right,...,68,74,75,41,50,43,13,13,6,9
1,Piero Hincapié,Ecuador,19,76,84,Bayer 04 Leverkusen,CB,€15M,€21K,Left,...,53,33,74,78,79,75,9,15,10,9
2,Wilfried Gnonto,Italy,17,64,83,FC Zürich,ST,€1.5M,€950,Right,...,59,63,58,26,21,17,11,9,10,13
3,Gareth Bale,Wales,31,81,81,Real Madrid CF,RW,€21.5M,€170K,Left,...,77,76,85,50,53,50,15,15,11,6
4,Pablo Martín Páez Gavira,Spain,16,77,89,FC Barcelona,CM,€24M,€14K,Right,...,81,51,74,59,66,62,8,10,11,13


In [17]:
df_multi_thread["crossing"]

0      63
1      55
2      48
3      85
4      64
       ..
115    70
116    48
117    63
118    59
119    64
Name: crossing, Length: 120, dtype: object

In [ ]:
p_url = "https://sofifa.com/player/237712/valentin-castellanos/220056/"
response = requests.get(p_url)
soup = BeautifulSoup(response.content, "html.parser")
blocks = soup.findAll("div", {"class": "col-12"})
info = blocks[0]
stats = blocks[1]
#[extract_info(tr) for tr in trs]

In [ ]:
soup.findAll("div", {"class": "col-12"})[0].findAll("div", {"class":"block-quarter"})[4].find("ul").findAll("li")[0]
#profile = n[4]
#list_features = profile.find("ul").findAll("li")
#list_features[4].text.strip()[9:]

In [ ]:
result = {}
for i in range(len(list_features)):
    print(list_features[i].text.split())

In [2]:
# helper method for extracting data of the player

def extract_info(tr):
    base = "https://sofifa.com/"
    link = base + tr.select('td.col-name')[0].find("a").get("href")
    return {
        "name": tr.select('td.col-name')[0].find("a").get("aria-label"),
        "country": tr.select('td.col-name')[0].find("img").get("title"),
        "age": tr.select('td.col.col-ae')[0].text.strip(),
        "overall": tr.select('td.col.col-oa')[0].text.strip(),
        "potential": tr.select('td.col.col-pt')[0].text.strip(),
        "club": tr.select("td.col-name")[1].find("a").text,
        "best_position": tr.select('td.col-name')[0].find("span").text,
        "value": tr.select('td.col.col-vl')[0].text.strip(),
        "wage": tr.select('td.col.col-wg')[0].text.strip(),
        "stats": extract_stats(link)
    }


# helper method for extracting stats of single player

def extract_stats(link):
    new_res = requests.get(link)
    new_soup = BeautifulSoup(new_res.content, "html.parser")
    stats_block = new_soup.find_all("div", {"class": "center"})[5].findAll("div", {"class": "block-quarter"})
    profile = new_soup.findAll("div", {"class": "col-12"})[0].findAll("div", {"class":"block-quarter"})[4].find("ul").findAll("li")
    return {"preferred_foot" : profile[0].text.split()[1][4:],
            "weak_foot" : profile[1].text.split()[0],
            "skill_move": profile[2].text.split()[0],
            "work_rate" : profile[4].text.strip()[9:],
            "attrs": extract_deep(stats_block)
            }

def extract_deep(stats_block):
    return {"Att": extract_att(stats_block[0]),
            "Skill": extract_skill(stats_block[1]), 
            "Move": extract_move(stats_block[2]),
            "Power": extract_pow(stats_block[3]),
            "Mentality": extract_mentality(stats_block[4]),
            "Defending": extract_def(stats_block[5]),
            "Goalkeep": extract_goalkeep(stats_block[6])
            }

In [1]:
def extract_att(att):
    return {"crossing": att.findAll("li")[0].find("span").text.strip(),
            "Finishing": att.findAll("li")[1].find("span").text.strip(),
            "Heading Accuracy":  att.findAll("li")[2].find("span").text.strip(),
            "Short passing": att.findAll("li")[3].find("span").text.strip(),
            "Volleys": att.findAll("li")[4].find("span").text.strip()
            }


def extract_skill(ski):
    return {"Dribbling": ski.findAll("li")[0].find("span").text.strip(),
            "Curve": ski.findAll("li")[1].find("span").text.strip(),
            "Fk Accuracy": ski.findAll("li")[2].find("span").text.strip(),
            "Long Passing": ski.findAll("li")[3].find("span").text.strip(),
            "Ball Control": ski.findAll("li")[4].find("span").text.strip()
            }


def extract_move(mov):
    return {"Acceleration": mov.findAll("li")[0].find("span").text.strip(),
            "Sprint Speed": mov.findAll("li")[1].find("span").text.strip(),
            "Agility": mov.findAll("li")[2].find("span").text.strip(),
            "Reactions": mov.findAll("li")[3].find("span").text.strip(),
            "Balance": mov.findAll("li")[4].find("span").text.strip()
            }


def extract_pow(pow):
    return {"Shot Power": pow.findAll("li")[0].find("span").text.strip(),
            "Jumping": pow.findAll("li")[1].find("span").text.strip(),
            "Stamina": pow.findAll("li")[2].find("span").text.strip(),
            "Strength": pow.findAll("li")[3].find("span").text.strip(),
            "Long Shots": pow.findAll("li")[4].find("span").text.strip()
            }


def extract_mentality(men):
    return {"Aggression": men.findAll("li")[0].find("span").text.strip(),
            "Interceptions": men.findAll("li")[1].find("span").text.strip(),
            "Positioning": men.findAll("li")[2].find("span").text.strip(),
            "Vision": men.findAll("li")[3].find("span").text.strip(),
            "Penalties": men.findAll("li")[4].find("span").text.strip(),
            "Composure": men.findAll("li")[5].find("span").text.strip()
            }


def extract_def(defe):
    return {"Defensive Awareness": defe.findAll("li")[0].find("span").text.strip(),
            "Standing Tackle": defe.findAll("li")[1].find("span").text.strip(),
            "Sliding Tackle": defe.findAll("li")[2].find("span").text.strip()
            }


def extract_goalkeep(gk):
    return {"Diving": gk.findAll("li")[0].find("span").text.strip(),
            "Handling": gk.findAll("li")[1].find("span").text.strip(),
            "Kicking": gk.findAll("li")[2].find("span").text.strip(),
            "Positioning": gk.findAll("li")[3].find("span").text.strip(),
            "Reflexes": gk.findAll("li")[4].find("span").text.strip()
            }

In [5]:
jsonDat = "../data/raw/player_raw_data.json"
tb_name = "FIFA22"
dynamodb = boto3.resource("dynamodb", region_name="us-west-2")
table = dynamodb.Table(tb_name)
#n = 0
with open(jsonDat, "r", encoding="utf-8") as json_file:
    players = json.load(json_file)
    for player in players:
        name = player["name"]
        country = player["country"]
        age = player["age"]
        overall = player["overall"]
        potential = player["potential"]
        club = player["club"]
        best_position = player["best_position"]
        value = player["value"]
        wage = player["wage"]
        preferred_foot = player["preferred_foot"]
        weak_foot = player["weak_foot"]
        skill_move = player["skill_move"]
        work_rate = player["work_rate"]
        Crossing = player["Crossing"]
        Finishing = player["Finishing"]
        Heading_accuracy = player["Heading Accuracy"]
        Short_passing = player["Short passing"]
        Volleys = player["Volleys"]
        Dribbling = player["Dribbling"]
        Curve = player["Curve"]
        Fk_Accuracy = player["Fk Accuracy"]
        Long_Passing = player["Long Passing"]
        Ball_Control = player["Ball Control"]
        Acceleration = player["Acceleration"]
        Sprint_Speed = player["Sprint Speed"]
        Agility = player["Agility"]
        Reactions = player["Reactions"]
        Balance = player["Balance"]
        Shot_Power = player["Shot Power"]
        Jumping = player["Jumping"]
        Stamina = player["Stamina"]
        Strength = player["Strength"]
        Long_Shots = player["Long Shots"]
        Aggression = player["Aggression"]
        Interceptions = player["Interceptions"]
        Positioning = player["Positioning"]
        Vision = player["Vision"]
        Penalties = player["Penalties"]
        Composure = player["Composure"]
        Defensive_Awareness = player["Defensive Awareness"]
        Standing_Tackle = player["Standing Tackle"]
        Sliding_Tackle = player["Sliding Tackle"]
        Diving = player["Diving"]
        Handling = player["Handling"]
        Kicking = player["Kicking"]
        Reflexes = player["Reflexes"]
        #n = n + 1
        #print("Adding detail: ", name, club, n)

        table.put_item(
            Item={
                "name": name,
                "country": country,
                "age": age,
                "overall": overall,
                "potential": potential,
                "club": club,
                "best_position": best_position,
                "value": value,
                "wage": wage,
                "preferred_foot" : preferred_foot,
                "weak_foot" : weak_foot,
                "skill_move" : skill_move,
                "work_rate" : work_rate,
                "Crossing": Crossing,
                "Finishing": Finishing,
                "Heading Accuracy": Heading_accuracy,
                "Short passing": Short_passing,
                "Volleys": Volleys,
                "Dribbling": Dribbling,
                "Curve": Curve,
                "Fk Accuracy": Fk_Accuracy,
                "Long Passing": Long_Passing,
                "Ball Control": Ball_Control,
                "Acceleration": Acceleration,
                "Sprint Speed": Sprint_Speed,
                "Agility": Agility,
                "Reactions": Reactions,
                "Balance": Balance,
                "Shot Power": Shot_Power,
                "Jumping": Jumping,
                "Stamina": Stamina,
                "Strength": Strength,
                "Long Shots": Long_Shots,
                "Aggression": Aggression,
                "Interceptions": Interceptions,
                "Positioning": Positioning,
                "Vision": Vision,
                "Penalties": Penalties,
                "Composure": Composure,
                "Defensive Awareness": Defensive_Awareness,
                "Standing Tackle": Standing_Tackle,
                "Sliding Tackle": Sliding_Tackle,
                "Diving": Diving,
                "Handling": Handling,
                "Kicking": Kicking,
                "Reflexes": Reflexes
            }
        )

## Prototype

- Refactor the Scraper class, so that it has different branch of statements to handle either scrap surface only or scrap deep only

In [10]:
file_path = "../data/raw/player_raw_data.csv"
df = pd.read_csv(file_path)
# dynamodb = boto3.resource("dynamodb", region_name="us-west-2")
# table = dynamodb.Table("Testing")
# with table.batch_writer() as bw:
#     for i, record in enumerate(df.to_dict("records")):
#         bw.put_item(Item=record)

In [ ]:
pls = []
url = "https://sofifa.com/players?offset="
urls = [url + str(offset) for offset in range(0, 30, 10)]
for url in urls:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    tbody = soup.find("tbody", {"class": "list"})
    trs = tbody.findAll("tr")
    pls.append([extract_info(tr) for tr in trs])

####

first_url = pls[0][0]["link"]
first_url
r = requests.get(first_url)
sou = BeautifulSoup(r.content, "html.parser")
tb = soup.find_all("div", {"class": "center"})[5]
blo = tb.findAll("div", {"class" : "block-quarter"})
type(blo)

###

url = "https://sofifa.com/player/27488/jan-vennegoor-of-hess/120002/"
new_res = requests.get(url)
soup = BeautifulSoup(new_res.content, "html.parser")
tbody = soup.find_all("div", {"class": "center"})[5]
stats_block = tbody.findAll("div", {"class": "block-quarter"})

